In [2]:
%%html

<!-- Just run this cell to provide style to the notebook-->
<!-- This is a code to add style to the notebook, it is based on a .css on GitHub http://bit.ly/1Bf5Hft -->

<style>

html {
  font-size: 62.5% !important; }
body {
  font-size: 1.5em !important; /* currently ems cause chrome bug misinterpreting rems on body element */
  line-height: 1.6 !important;
  font-weight: 400 !important;
  font-family: "Raleway", "HelveticaNeue", "Helvetica Neue", Helvetica, Arial, sans-serif !important;
  color: #222 !important; }

div{ border-radius: 0px !important;  }
div.CodeMirror-sizer{ background: rgb(244, 244, 248) !important; }
div.input_area{ background: rgb(244, 244, 248) !important; }

div.out_prompt_overlay:hover{ background: rgb(244, 244, 248) !important; }
div.input_prompt:hover{ background: rgb(244, 244, 248) !important; }

h1, h2, h3, h4, h5, h6 {
  color: #333 !important;
  margin-top: 0 !important;
  margin-bottom: 2rem !important;
  font-weight: 300 !important;
    text-decoration: underline;
}
h1 { font-size: 4.0rem !important; line-height: 1.2 !important;  letter-spacing: -.1rem !important;}
h2 { font-size: 3.6rem !important; line-height: 1.25 !important; letter-spacing: -.1rem !important; }
h3 { font-size: 3.0rem !important; line-height: 1.3 !important;  letter-spacing: -.1rem !important; }
h4 { font-size: 2.4rem !important; line-height: 1.35 !important; letter-spacing: -.08rem !important; }
h5 { font-size: 1.8rem !important; line-height: 1.5 !important;  letter-spacing: -.05rem !important; }
h6 { font-size: 1.5rem !important; line-height: 1.6 !important;  letter-spacing: 0 !important; }
    
@media (min-width: 550px) {
  h1 { font-size: 5.0rem !important; }
  h2 { font-size: 4.2rem !important; }
  h3 { font-size: 3.6rem !important; }
  h4 { font-size: 3.0rem !important; }
  h5 { font-size: 2.4rem !important; }
  h6 { font-size: 1.5rem !important; }
}

p {
    margin-top: 0 !important;
    margin-bottom: 1rem !important;
    text-align: justify;
    text-justify: inter-word;
    line-height: 1.5 !important;
    font-size: 1.2em !important;
    font-family: "Raleway", "HelveticaNeue", "Helvetica Neue", Helvetica, Arial, sans-serif !important;
}
  
a {
  color: #1EAEDB !important; }
a:hover {
  color: #0FA0CE !important; }
  
code {
  padding: .2rem .5rem !important;
  margin: 0 .2rem !important;
  font-size: 90% !important;
  white-space: nowrap !important;
  background: #F1F1F1 !important;
  border: 1px solid #E1E1E1 !important;
  border-radius: 4px !important; }
pre > code {
  display: block !important;
  padding: 1rem 1.5rem !important;
  white-space: pre !important; }
  
button{ border-radius: 0px !important; }
.navbar-inner{ background-image: none !important;  }
select, textarea{ border-radius: 0px !important; }
    
#Top_Header {
    background-image: url(https://lh3.googleusercontent.com/-shldIGCRFdbscyKdAvFOATYt3wRk3SWzxQ-OSLQvvGQQEekDY2nrNYhcTl9nB3gcx_t=w300), url(https://lh3.googleusercontent.com/-shldIGCRFdbscyKdAvFOATYt3wRk3SWzxQ-OSLQvvGQQEekDY2nrNYhcTl9nB3gcx_t=w300);
    background-size: contain;
    background-repeat: no-repeat;
    background-position: left, right;
}

.output {
    display: flex;
    align-items: center;
    text-align: center;
}
    
</style>
    
<script> 
$( document ).ready(function () {
    $("div#notebook-container").children().first().hide();
});
</script>

In [1]:
%%javascript
//Execute this code to show the styling of the document
$("div#notebook-container").children().first().show();

<IPython.core.display.Javascript object>

<div id="Top_Header">
    <center>
        <h1>Dog Breed Identification</h1>
        <h3>Deep Learning: CNN from Scratch</h3>
        <h6>Andreu Masdeu - <i>TFG 2018</i></h6>
    </center>
</div>

<div id="Introduction">
    <h2>Introduction</h2>
    <p>This notebook is an approach to the <i>Kaggle</i> competition <strong><a href="https://www.kaggle.com/c/dog-breed-identification">Dog Breed Identification</a></strong> using <i>Convolutional Neural Networks</i>.
    </p>
    <p>This is not the ideal approach since we would need a huge dataset and computional power. But it will be a good example for comparing performances with the Transfer Learning notebook. </p>
</div>

<h1> Importing </h1>

<p> We import pandas, numpy, keras and some of its layers and methods, required for defining the network architecture. Furthermore, we import cv2, a powerful package useful for image processing and computer vision. </p>

In [ ]:
import pandas as pd
import numpy as np

import keras
from keras.models import Model, Sequential
from keras.layers import Dense, Embedding, Input, Conv2D, MaxPooling2D, Flatten, Dropout, LeakyReLU
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers.normalization import BatchNormalization


import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode()
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
import cv2
from tqdm import tqdm
import time
import random

<h1> Auxiliar functions </h1>

<p> This cell includes useful functions. Get_pixels transforms a picture into an array of numbers with the desired dimension. Normalize, as its name suggests, normalizes those arrays into an interval [0,1]. Finally, show_image creates a window showing the picture given and it is useful to see differences between normalized and normal pictures. </p>

In [ ]:
def get_pixels(dim, path ):
    image = cv2.imread(path, 1)
    resized = cv2.resize(image, (dim, dim))
    return resized

def show_image(img, dim=224):
    #img = img.reshape((dim,dim,3))
    plt.imshow(img, cmap='gray')
    plt.show()

def normalize(img):
    img  = img/255
    return np.array(img, dtype= 'float32')

# Paths and labels

Labels for each photo are contained in a csv file. The following lines store in a pandas dataframe the root path and label for each picture.

In [ ]:
trainfolder = "train/train/"
paths_labels = pd.read_csv('labels.csv/labels.csv')
paths_labels['image_path'] = paths_labels.apply(lambda x: (trainfolder + x["id"] + ".jpg" ), axis=1)

# Load Data

In [ ]:
DIM = int(input("Write the number of pixels desired "))

In [ ]:
lst = []
for path in tqdm(paths_labels['image_path']):
    lst.append(normalize(get_pixels(DIM,path)))
    

labels = pd.get_dummies(paths_labels, columns=['breed'])

In [ ]:
data = np.array(lst)
del lst

In [ ]:
dropped = []
for i in range(0,2):
    dropped.append(labels.columns[i])
    
labels = labels.drop(dropped, axis=1, inplace=False)

In [ ]:
dic = {}
num_classes = len(labels.columns)
for i in range(0, num_classes-1):
    l = list(labels.columns[i])
    l[0:6] = []
    elem2 = "".join(l)
    dic[labels.columns[i]] = elem2
    
labels = labels.rename(dic, axis = "columns")
dogs_list = labels.columns

In [ ]:
labels = labels.values

# Dataset distribution

In [ ]:
num_for_breed = [0 for x in dogs_list]
for label in labels:
    num_for_breed[label.argmax()] += 1

In [ ]:
trace0 = go.Bar(
    x=dogs_list,
    y=num_for_breed,
    marker=dict(
        color='rgb(158,202,225)',
        line=dict(
            color='rgb(8,48,107)',
            width=1.5,
        )
    ),
    opacity=0.6
)

data_plot = [trace0]
layout = go.Layout(
    title='Breed distribution',
)

fig = go.Figure(data=data_plot, layout=layout)
py.iplot(fig, filename='text-hover-bar')

<h1> Neural Network architecture </h1>

<p> Here we define our architecture. We opted for 3 convolutional layers with leaky relu activation, each one of them followed with a pooling layer and dropout. Finally, an undefined number of fully connected layers and the output layer complete the structure. For the hidden layers we use leaky relu as activation, but for the output we use softmax. We will predict a probability for each class and we want probabilities to sum 1. </p>

<p> Then, we compile this model using categorical cross entropy as the loss function, and we return it by calling conv_model().<p>
    
<p> This is rather a simple method, compared to the ones used in the transfer learning notebook. <p>

In [ ]:
def conv_model( nhid = 3, strides = 1, kernel = 3, drop = 0.5, drop2 = 0, alpha = 0.05 ):
    model = Sequential()
    
    #First Conv Layer
    
    model.add(Conv2D(input_shape = (DIM,DIM,3) , filters=32, kernel_size=kernel, strides = strides ))
    model.add(LeakyReLU(alpha=alpha))

    model.add(MaxPooling2D(pool_size=2))
    
    model.add(Dropout(drop))
    
    #Second Conv Layer
    
    model.add(Conv2D(filters=32, kernel_size=kernel, strides = strides ))
    model.add(LeakyReLU(alpha=alpha))

    model.add(MaxPooling2D(pool_size=2))
    
    model.add(Dropout(drop))
    
    #Third Conv Layer
    
    model.add(Conv2D(filters=64, kernel_size=kernel, strides = strides ))
    model.add(LeakyReLU(alpha=alpha))

    model.add(MaxPooling2D(pool_size=2))
    
    #Fourth Conv Layer
    
    model.add(Conv2D(filters=128, kernel_size=kernel, strides = strides ))
    model.add(LeakyReLU(alpha=alpha))

    model.add(MaxPooling2D(pool_size=2))
    
     #Fifth Conv Layer
    
    model.add(Conv2D(filters=128, kernel_size=kernel, strides = strides ))
    model.add(LeakyReLU(alpha=alpha))

    model.add(MaxPooling2D(pool_size=2))
    
    model.add(Flatten())
    
    #Fully connected layers
    
    for i in range(0,nhid):
        model.add(Dense(int(256/(2 ** i))))
        model.add(LeakyReLU(alpha = alpha))
        model.add(BatchNormalization())
        model.add(Dropout(drop2))

    model.add(Dense(120, activation = 'softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['accuracy'])
    return model

In [ ]:
model = conv_model(nhid = 1, drop = 0.0, drop2  = 0.0)

In [ ]:
model.summary()

<h1> Training </h1>

<p> The following cells implement the training process. Using the method .fit on the model and setting hyperparameters for the training, as the number of epochs and batch sizes, keras starts the backpropagation and gradient descent optimization. Before that, we implemented the dataset split using the train_test_split function from sklearn. It creates a random split setting aside a 10% of the pictures.</p>

In [ ]:
data_train, data_test, labels_train, labels_test = train_test_split(data, labels, test_size=0.1, shuffle = True)

In [ ]:
history = model.fit(data_train, labels_train, batch_size = 32, epochs = 20, callbacks = [], 
                    validation_data = (data_test, labels_test), verbose = 2)

<h1> Validation </h1>

A straightforward way of checking our model's performance is by plotting accuracy and loss values.

In [ ]:
y_axis1 = history.history['acc']
y_axis2 = history.history['val_acc']
y_axis3 = history.history['loss']
y_axis4 = history.history['val_loss']
x_axis = list(range(1,21))

In [ ]:
trace1 = go.Scatter(x = x_axis, y = y_axis1, mode = 'lines+markers', name = 'Training accuracy')

trace2 = go.Scatter(x = x_axis, y = y_axis2, mode = 'lines+markers', name = 'Validation accuracy')

data1 = [trace1, trace2]

py.iplot(data1, filename='basic-line')

In [ ]:
trace2 = go.Scatter(x = x_axis, y = y_axis3, mode = 'lines+markers', name = 'Training loss')

trace3 = go.Scatter(x = x_axis, y = y_axis4, mode = 'lines+markers', name = 'Validation loss')

data2 = [trace2, trace3]

py.iplot(data2, filename='basic-line')